# Import packages

In [1]:
# Packages
from IPython.core.display import HTML
import pandas as pd
import numpy as np
import xarray as xr
from datetime import datetime
import requests
import shutil
import time
import eumdac
import os

/Users/eugenerotherham/Desktop/Environments/Atheras_Analytics/lib/python3.9/site-packages/ecmwflibs/__init__.py:83: UserWarning: dlopen(/Users/eugenerotherham/Desktop/Environments/Atheras_Analytics/lib/python3.9/site-packages/ecmwflibs/_ecmwflibs.cpython-39-darwin.so, 2): Symbol not found: __ZNKSt3__115basic_stringbufIcNS_11char_traitsIcEENS_9allocatorIcEEE3strEv
  Referenced from: /Users/eugenerotherham/Desktop/Environments/Atheras_Analytics/lib/python3.9/site-packages/ecmwflibs/.dylibs/libproj.25.9.1.0.dylib (which was built for Mac OS X 12.0)
  Expected in: /usr/lib/libc++.1.dylib

  warnings.warn(str(e))


In [2]:
# Constants
consumer_key = '8fmYIf3P4ugcQJGZuNAF13i_fzwa'
consumer_secret =  'TM7zBHbXrL7ukrQ7ltGM2NplCcAa'
MAX_RETRIES = 10
raw_grib_destination_folder = '/Users/eugenerotherham/Documents/AtherasAnalytics/clouds-outage-prediction-main/eumetsat_lab/grib_staging_folder/'
results_destination_folder = '/Users/eugenerotherham/Documents/AtherasAnalytics/clouds-outage-prediction-main/eumetsat_lab/results/'

# Functions

In [3]:
def get_eumetsat_api_token():
    credentials = (consumer_key, consumer_secret)
    token = eumdac.AccessToken(credentials)
    try:
        print(f"This token '{token}' expires {token.expiration}")
    except requests.exceptions.HTTPError as exc:
        print(f"Error when tryng the request to the server: '{exc}'")
    return token

In [4]:
def get_collection(token, collection_id):
    datastore = eumdac.DataStore(token)
    selected_collection = datastore.get_collection(collection_id)
    
    try:
        print(selected_collection.search_options)
    except eumdac.datastore.DataStoreError as error:
        print(f"Error related to the data store: '{error.msg}'")
    except eumdac.collection.CollectionError as error:
        print(f"Error related to the collection: '{error.msg}'")
    except requests.exceptions.RequestException as error:
        print(f"Unexpected error: {error}")
    return selected_collection, datastore
    

In [5]:
def get_product_list(selected_collection, start_date, end_date):
    products = selected_collection.search(
            dtstart=start_date, 
            dtend=end_date)
    try:
        print(f'Found Datasets: {len(products)} datasets for the given time range.')   
    except eumdac.collection.CollectionError as error:
        print(f"Error related to the collection: '{error.msg}'")
    except requests.exceptions.RequestException as error:
        print(f"Unexpected error: {error}")

    product_list = []

    for product in products:
        retries = 0
        while retries < MAX_RETRIES:
            try:
                product_list.append(str(product))
                break
            except eumdac.product.ProductError as error:
                print(f"Error related to the product: '{error.msg}'")
            except requests.exceptions.RequestException as error:
                print(f"Unexpected error: {error}")
            
    return product_list

In [6]:
def get_station_bbox():

    # List all stations of interest just inpout code and their coordinates
    locations_data_df = {
            'station_code': ['matm', 'grsm'],
            'lon/lat': [(16.7046, 40.6486), (6.9216, 43.7546)]
        }

    # Area to average cloud coverage over (in degrees)
    scan_radius = 0.1

    station_bbox_list = []

    for index, row in pd.DataFrame(locations_data_df).iterrows():
        station_code = row['station_code']
        coord = row['lon/lat']
        
        lat_min = coord[1] - scan_radius
        lat_max = coord[1] + scan_radius
        lon_min = coord[0] - scan_radius
        lon_max = coord[0] + scan_radius
        bbox = [lat_min, lat_max, lon_min, lon_max]
        
        station_bbox_list.append({
            'station_code': station_code,
            'coord': coord,
            'bbox': bbox
        })

    station_bbox_df = pd.DataFrame(station_bbox_list)

    return station_bbox_df

In [7]:
def download_products(product_id, collection_id, datastore):
    
    selected_product = datastore.get_product(product_id=product_id, collection_id=collection_id)

    retries = 0
    grib_file_name = product_id + '.grb'
    
    while retries < MAX_RETRIES:
        try:
            with selected_product.open(entry=grib_file_name) as fsrc, \
                    open(raw_grib_destination_folder + grib_file_name, mode='wb') as fdst:
                shutil.copyfileobj(fsrc, fdst)
                break
        except eumdac.product.ProductError as error:
            if error.extra_info['status'] >= 500:
                wait = retries * 10
                time.sleep(wait)
                print(f"Attempt {retries}/{MAX_RETRIES}: {error}")
                retries += 1
            else:
                print(
                    f"Error related to the product '{selected_product}' while trying to download it: '{error.msg}'")
        except requests.exceptions.ConnectionError as error:
            wait = retries * 10
            time.sleep(wait)
            print(f"Attempt {retries}/{MAX_RETRIES}: {error}")
            retries += 1
        except requests.exceptions.RequestException as error:
            if error.extra_info['status'] >= 500:
                wait = retries * 10
                time.sleep(wait)
                print(f"Attempt {retries}/{MAX_RETRIES}: {error}")
                retries += 1
            else:
                print(f"Unexpected error: {error}")
    
    return grib_file_name


In [8]:
def get_date(grib_file_name):
    
    # Find the position of the 5th dash and the first full stop
    fifth_dash_pos = -1
    for _ in range(5):
        fifth_dash_pos = grib_file_name.find('-', fifth_dash_pos + 1)
    first_dot_pos = grib_file_name.index('.')
    
    # Extract the datetime part
    datetime_part = grib_file_name[fifth_dash_pos + 1 : first_dot_pos]
    #product_date = datetime.strptime(datetime_part, '%Y%m%d%H%M%S')

    return datetime_part 

# Main

In [9]:

import datetime

current_date = datetime.datetime(2023, 6, 1)

while current_date < datetime.datetime(2023, 8, 1):
    formatted_date = current_date.strftime('%Y-%m-%d')
    start_date = formatted_date
    end_date = (current_date + datetime.timedelta(days=31)).replace(day=1) - datetime.timedelta(days=1)
    print(f'start date: {start_date}')
    print(f'end date: {end_date}')

    # go

    current_date = current_date.replace(day=1) + datetime.timedelta(days=31)  # Move to the first day of the next month

start date: 2023-06-01
end date: 2023-06-30 00:00:00
start date: 2023-07-02
end date: 2023-07-31 00:00:00


In [10]:
# Initial parameters
import datetime

collection_id = 'EO:EUM:DAT:MSG:CLM'
start_date = datetime.datetime(2023, 6, 1, 0, 0)
end_date = datetime.datetime(2023, 8, 1, 0, 0)

token = get_eumetsat_api_token()

selected_collection, datastore = get_collection(token, collection_id)

product_list = get_product_list(selected_collection, start_date, end_date)

station_bbox_df = get_station_bbox()

This token 'ff3ed47f-dabc-39f0-87c1-d479eb491a54' expires 2023-08-24 23:31:17.727020
{'bbox': {'title': 'Inventory which has a spatial extent overlapping this bounding box', 'options': []}, 'geo': {'title': 'Inventory which has a spatial extent overlapping this Well Known Text geometry', 'options': []}, 'title': {'title': 'Can be used to define a wildcard search on the product title (product identifier), use set notation as OR and space as AND operator between multiple search terms', 'options': [None]}, 'sat': {'title': 'Mission / Satellite', 'options': ['MSG2', 'MSG3', 'MSG4', 'MSG1']}, 'type': {'title': 'Product Type', 'options': ['MSGCLMK']}, 'dtstart': {'title': 'Temporal Start', 'options': []}, 'dtend': {'title': 'Temporal End', 'options': []}, 'publication': {'title': 'publication date', 'options': []}, 'sort': {'title': 'SRU sort keys, see OpenSearch Extension for EO for details', 'options': ['publicationDate', 'start,time']}, 'set': {'title': 'element set to return for each hit

In [11]:
# Create empty weather DataFrames for each station using dictionary comprehension
station_weather_dataframes = {row['station_code']: pd.DataFrame(columns=['time', 'cloud_cov']) for index, row in station_bbox_df.iterrows()}

total_products = len(product_list)
for i, product_id in enumerate(product_list, 1):  # Start index from 1 to match human counting

    print(f"Parsing GRIB: {i}/{total_products}")

    try:
        # Download dataset
        grib_file_name = download_products(product_id, collection_id, datastore) 
        product_date = get_date(grib_file_name)

        # Open dataset
        ds=xr.open_dataset(raw_grib_destination_folder+grib_file_name,engine='cfgrib',backend_kwargs={'indexpath': ''})

        # Prepare and format data for plotting
        data_array = ds['p260537']
        latitude = data_array['latitude']
        longitude = data_array['longitude']
        longitude = (longitude + 180) % 360 - 180
        longitude = longitude[::-1]
        latitude = latitude[::-1]
        # Convert to pandas series
        data_series = pd.Series(data_array, name='cloud_mask')
        latitude_series = pd.Series(latitude, name='latitude')
        longitude_series = pd.Series(longitude, name='longitude')
        # Convert to pandas dataframe
        raw_df = pd.concat([latitude_series, longitude_series, data_series], axis=1)
        
        # In current dataset extract cloud cov from each station    and add to station datframe save to txt file
        for index, row in station_bbox_df.iterrows():      
            bbox = row['bbox']
            station_code = row['station_code']
            filtered_df = raw_df[(raw_df['latitude'] >= bbox[0]) & (raw_df['latitude'] <= bbox[1]) & (raw_df['longitude'] >= bbox[2]) & (raw_df['longitude'] <= bbox[3])]
            
            cloudy_sum = (filtered_df['cloud_mask'] == 2).sum()
            total = len(filtered_df['cloud_mask'])
            station_cloud_cov = (cloudy_sum / total)*100

            station_cloud_cov_to_be_added = {'time': product_date, 'cloud_cov': station_cloud_cov}
            station_weather_dataframes[station_code] = pd.concat([station_weather_dataframes[station_code], pd.DataFrame([station_cloud_cov_to_be_added])], ignore_index=True)
        
        # Delete the GRIB file from the file system if it exists
        ds.close()
        file_path = os.path.join(raw_grib_destination_folder, grib_file_name)
        if os.path.exists(file_path):
            os.remove(file_path)
    except BaseException as e:
        print(f"An error occurred: {type(e).__name__} - {e}")
        continue
    
for index, row in station_bbox_df.iterrows():   
    station_code = row['station_code']
    station_weather_dataframes[station_code].to_csv(f"{results_destination_folder}{station_code}_eumetsat_{start_date}_{end_date}_df", sep='\t')    


Parsing GRIB: 1/5857
Parsing GRIB: 2/5857
Parsing GRIB: 3/5857
Parsing GRIB: 4/5857
Parsing GRIB: 5/5857
Parsing GRIB: 6/5857
Parsing GRIB: 7/5857
Parsing GRIB: 8/5857
Parsing GRIB: 9/5857
Parsing GRIB: 10/5857
Parsing GRIB: 11/5857
Parsing GRIB: 12/5857
Parsing GRIB: 13/5857
Parsing GRIB: 14/5857
Parsing GRIB: 15/5857
Parsing GRIB: 16/5857
Parsing GRIB: 17/5857
Parsing GRIB: 18/5857
Parsing GRIB: 19/5857
Parsing GRIB: 20/5857
Parsing GRIB: 21/5857
Parsing GRIB: 22/5857
Parsing GRIB: 23/5857
Parsing GRIB: 24/5857
Parsing GRIB: 25/5857
Parsing GRIB: 26/5857
Parsing GRIB: 27/5857
Parsing GRIB: 28/5857
Parsing GRIB: 29/5857
Parsing GRIB: 30/5857
Parsing GRIB: 31/5857
Parsing GRIB: 32/5857
Parsing GRIB: 33/5857
Parsing GRIB: 34/5857
Parsing GRIB: 35/5857
Parsing GRIB: 36/5857
Parsing GRIB: 37/5857
Parsing GRIB: 38/5857
Parsing GRIB: 39/5857
Parsing GRIB: 40/5857
Parsing GRIB: 41/5857
Parsing GRIB: 42/5857
Parsing GRIB: 43/5857
Parsing GRIB: 44/5857
Parsing GRIB: 45/5857
Parsing GRIB: 46/58

# Join monthly files into yearly and check for errors

In [1]:
# code to joining monthly data into yearly data file

import os
import pandas as pd

def join_csv_files(folder_path, output_file):
    all_dataframes = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)
            all_dataframes.append(df)

    if all_dataframes:
        # Reset the index for each dataframe before concatenating
        for idx, df in enumerate(all_dataframes):
            all_dataframes[idx] = df.reset_index(drop=True)

        combined_df = pd.concat(all_dataframes, ignore_index=True)
        combined_df.to_csv(output_file, index=False)
        print(f"Successfully joined {len(all_dataframes)} CSV files into {output_file}")
    else:
        print("No CSV files found in the specified folder.")

# Example usage:
folder_path = "/Users/eugenerotherham/Documents/AtherasAnalytics/clouds-outage-prediction-main/eumetsat_lab/stations/matm"
output_file = "/Users/eugenerotherham/Documents/AtherasAnalytics/clouds-outage-prediction-main/eumetsat_lab/stations/matm_cloud_2022-06_2023-08.txt"
join_csv_files(folder_path, output_file)



Successfully joined 5 CSV files into /Users/eugenerotherham/Documents/AtherasAnalytics/clouds-outage-prediction-main/eumetsat_lab/stations/matm_cloud_2022-06_2023-08.txt


In [ ]:
# check time differences and format


'''
# check if there are any gaps in the data (time differences greater than 15 minutes)
cloud_df['time'] = pd.to_datetime(cloud_df['time'])
time_diff = cloud_df['time'].diff()


cloud_df['time_difference'] = time_diff
cloud_df['time_difference_exceeds_threshold'] = time_diff > pd.Timedelta(minutes=15)

if any(cloud_df['time_difference_exceeds_threshold']):
    print("Intervals greater than 15 minutes:")
    # print(cloud_df[cloud_df['time_difference_exceeds_threshold']])
    # Calculate the length of cloud_df where time differences exceed the threshold
    length_exceeding_threshold = len(cloud_df[cloud_df['time_difference_exceeds_threshold']])
    print(f"The length of cloud_df with time differences exceeding 15 minutes is: {length_exceeding_threshold}")
else:
    print("The dataset has complete and consistent time intervals (15 minutes apart).")

'''
